# Лабораторная работа №4 <br>
Цель работы: познакомиться с библиотекой Optuna. Оптимизировать алгоритм градиентого спуска.

In [8]:
import numpy as np
import optuna
import sklearn.datasets
import sklearn.linear_model
import sklearn.metrics
import sklearn.model_selection
from sklearn.model_selection import train_test_split
import logging
import lightgbm as lgb
import numpy as np
import plotly

import sys
#Графики
from optuna.visualization import plot_contour
from optuna.visualization import plot_edf
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_slice

Запишем функции

In [9]:
def func1(x, y):      # Функция Матьяса  -10:10
    return 0.26 * (x*x + y*y) - 0.48*x*y

def objective_f1(trial):
    x = trial.suggest_float('x', -10, 10)
    y = trial.suggest_float('y', -10, 10)
    return func1(x, y)

In [10]:
def func2(x, y): # Функция Химмельблау -5:5 
    return (x * x + y - 11)**2 + (x + y * y - 7)**2

def objective_f2(trial):
    x = trial.suggest_float('x', -5, 5)
    y = trial.suggest_float('y', -5, 5)
    return func2(x, y)


Запустим оптисизацию для первой функции

In [11]:
study_f1 = optuna.create_study()
study_f1.optimize(objective_f1, n_trials=100)

[I 2022-12-21 15:06:29,349] A new study created in memory with name: no-name-9492e8b8-6c6e-45a6-9a42-f8edf208e969
[I 2022-12-21 15:06:29,361] Trial 0 finished with value: 44.52596004610007 and parameters: {'x': 8.786662911390806, 'y': -4.531732748627732}. Best is trial 0 with value: 44.52596004610007.
[I 2022-12-21 15:06:29,367] Trial 1 finished with value: 24.102315876459663 and parameters: {'x': 1.0580222554665006, 'y': -8.64290780079158}. Best is trial 1 with value: 24.102315876459663.
[I 2022-12-21 15:06:29,380] Trial 2 finished with value: 0.845866526592117 and parameters: {'x': 3.581189764652562, 'y': 2.141172835079935}. Best is trial 2 with value: 0.845866526592117.
[I 2022-12-21 15:06:29,384] Trial 3 finished with value: 6.8191980166362995 and parameters: {'x': -9.133568157832851, 'y': -4.704440382264261}. Best is trial 2 with value: 0.845866526592117.
[I 2022-12-21 15:06:29,389] Trial 4 finished with value: 5.204953718546491 and parameters: {'x': -9.102877822459725, 'y': -5.61

Выведем лучшие параметры

In [12]:
best_params_f1 = study_f1.best_params
found_x = best_params_f1["x"]
found_y = best_params_f1["y"]
print("Функция Матиаса:")
print(f"x: {found_x}, y: {found_y}, E: {func1(found_x, found_y)}")

Функция Матиаса:
x: -0.9071939746308828, y: -0.7444750531539125, E: 0.03389946762509288


Повторим для второй функции

In [13]:
study_f2 = optuna.create_study()
study_f2.optimize(objective_f2, n_trials=100)

[I 2022-12-21 15:06:32,505] A new study created in memory with name: no-name-b4a89c90-0cd4-4b4f-aad4-b9de3f20cdfc
[I 2022-12-21 15:06:32,516] Trial 0 finished with value: 173.31188072578476 and parameters: {'x': -4.960610928488231, 'y': -4.648268022321402}. Best is trial 0 with value: 173.31188072578476.
[I 2022-12-21 15:06:32,525] Trial 1 finished with value: 95.96443035119322 and parameters: {'x': -0.277293998004712, 'y': 1.8761659277892164}. Best is trial 1 with value: 95.96443035119322.
[I 2022-12-21 15:06:32,531] Trial 2 finished with value: 129.26831549883653 and parameters: {'x': 0.7551648725717364, 'y': 0.6578897745256977}. Best is trial 1 with value: 95.96443035119322.
[I 2022-12-21 15:06:32,536] Trial 3 finished with value: 82.12677398728712 and parameters: {'x': 0.3952683807659163, 'y': 2.0763748171024465}. Best is trial 3 with value: 82.12677398728712.
[I 2022-12-21 15:06:32,543] Trial 4 finished with value: 430.94019617471236 and parameters: {'x': 0.7885744992041595, 'y': 

In [14]:
best_params_f2 = study_f2.best_params
found_x = best_params_f2["x"]
found_y = best_params_f2["y"]
print("Функция Химмельблау:")
print(f"x: {found_x}, y: {found_y}, E: {func2(found_x, found_y)}")

Функция Химмельблау:
x: 2.952636143480007, y: 2.089081657233052, E: 0.13761880631639656


Оптимизируем SGD(Датасет wine)

In [16]:
def objective(trial):
    DS = sklearn.datasets.load_wine()
    classes = list(set(DS.target))
    train_x, valid_x, train_y, valid_y = sklearn.model_selection.train_test_split(
        DS.data, DS.target, test_size=0.25, random_state=0)
    alpha = trial.suggest_float("alpha", 1e-5, 1e-1, log=True)
    clf = sklearn.linear_model.SGDClassifier(alpha=alpha)
    for n in range(100):
        clf.partial_fit(train_x, train_y, classes=classes)
        intermediate_value = 1.0 - clf.score(valid_x, valid_y)
        trial.report(intermediate_value, n)
        if trial.should_prune():
            raise optuna.TrialPruned()
    return 1.0 - clf.score(valid_x, valid_y)

In [17]:
study = optuna.create_study(
    sampler=optuna.samplers.TPESampler(seed=2134),
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=10),
)
study.optimize(objective, n_trials=100, timeout=600)

[I 2022-12-21 15:06:35,497] A new study created in memory with name: no-name-2b1b435a-33a0-4cbd-bb38-167a3c655eaa
[I 2022-12-21 15:06:36,157] Trial 0 finished with value: 0.1777777777777778 and parameters: {'alpha': 4.631004897647501e-05}. Best is trial 0 with value: 0.1777777777777778.
[I 2022-12-21 15:06:36,803] Trial 1 finished with value: 0.24444444444444446 and parameters: {'alpha': 0.00016605788699992395}. Best is trial 0 with value: 0.1777777777777778.
[I 2022-12-21 15:06:37,471] Trial 2 finished with value: 0.4 and parameters: {'alpha': 4.869004143982021e-05}. Best is trial 0 with value: 0.1777777777777778.
[I 2022-12-21 15:06:38,098] Trial 3 finished with value: 0.19999999999999996 and parameters: {'alpha': 1.2268051341010467e-05}. Best is trial 0 with value: 0.1777777777777778.
[I 2022-12-21 15:06:38,584] Trial 4 finished with value: 0.48888888888888893 and parameters: {'alpha': 1.0764096236344291e-05}. Best is trial 0 with value: 0.1777777777777778.
[I 2022-12-21 15:06:39,10

In [18]:
plot_optimization_history(study)

In [19]:
plot_intermediate_values(study)

In [20]:
plot_parallel_coordinate(study)

In [22]:
plot_slice(study)

In [23]:
plot_param_importances(study)

In [24]:
plot_edf(study)